In [167]:
#Set up environment
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import time


###OPTIONAL: Uncomment code if you would like run a report already downloaded
##Change to your desired FilePath & FileName
#FilePathDay = 'Downloads'
#FileNameDay = 'order_test.csv'
#FilePathYear = 'Downloads'
#FileNameYear = 'trades_order_test.csv'

#Read csv file
#dfYear = pd.read_csv(FilePathYear+'/'+FileNameYear)
#dfDay = pd.read_csv(FilePathDay+'/'+FileNameDay)
###OPTIONAL: End of commented area


###OPTIONAL: Must comment below code if running commented code above
#Code to get the report
sendUrl = "https://ndcdyn.interactivebrokers.com/AccountManagement/FlexWebService/SendRequest?"
###REQUIRED: Must update token as it changes
token = "t=694266458393324010438090"
###REQUIRED: Must update if you want to use a different flex query
queryIdYear = "&q=944048"
version = "&v=3"

requestUrlYear = "".join([sendUrl, token, queryIdYear, version])
flexReqYear = requests.get(url=requestUrlYear)

if flexReqYear.status_code != 200:
  print("Error!")
  exit()
treeYear = ET.ElementTree(ET.fromstring(flexReqYear.content))
rootYear = treeYear.getroot()
getURLYear = rootYear.find('Url').text
refCodeYear = "&q=%s" % rootYear.find('ReferenceCode').text
receiveUrlYear = "".join([getURLYear, "?",token, refCodeYear, version])

print("Hold for Request.")
time.sleep(20)

receiveUrlYear = requests.get(url=receiveUrlYear, allow_redirects=True)

###REQUIRED: Must change report name before running or delete previous instance
ReportPathYear = 'Downloads'
ReportNameYear = 'StockReportYear.csv'

open(ReportPathYear+'/'+ReportNameYear, 'wb').write(receiveUrlYear.content)
print("First report obtained successfully")


###REQUIRED: Must update if you want to use a different flex query
queryIdDay = "&q=943961"

requestUrlDay = "".join([sendUrl, token, queryIdDay, version])
flexReqDay = requests.get(url=requestUrlDay)

if flexReqDay.status_code != 200:
  print("Error!")
  exit()
treeDay = ET.ElementTree(ET.fromstring(flexReqDay.content))
rootDay = treeDay.getroot()
getURLDay = rootDay.find('Url').text
refCodeDay = "&q=%s" % rootDay.find('ReferenceCode').text
receiveUrlDay = "".join([getURLDay, "?",token, refCodeDay, version])

print("Hold for Request.")
time.sleep(20)

receiveUrlDay = requests.get(url=receiveUrlDay, allow_redirects=True)

###REQUIRED: Must change report name before running or delete previous instance
ReportPathDay = 'Downloads'
ReportNameDay = 'StockReportDay.csv'

open(ReportPathDay+'/'+ReportNameDay, 'wb').write(receiveUrlDay.content)
print("Second report obtained successfully")


#Read csv file
dfYear = pd.read_csv(ReportPathYear+'/'+ReportNameYear)
dfDay = pd.read_csv(ReportPathDay+'/'+ReportNameDay)
###OPTIONAL: End of commented area

#Sets up combined file
dfYear.rename(columns={'TradePrice':'Price', 'IBCommission':'Commission'}, inplace=True)
df = pd.concat([dfYear, dfDay], ignore_index = True)
df['TradeDate'] = pd.to_datetime(df['TradeDate'])

#Drops any entries that are not STK
df.drop(df[df.AssetClass != 'STK'].index, inplace=True)

#Creates array with all stock companies
symbols = df['Symbol'].unique()
#Sets empty dictionary to hold dataframes
d = {}
#Empty dataframe for stocks to be printed
results = pd.DataFrame()
pairedColumns = ['AssetClass','Symbol','BuyDate','SellDate','Quantity','BuyPrice','SellPrice','BuyCommission',
                 'SellCommission','Profit','DaysHeld','ProfitsLost']
pairedResults = pd.DataFrame(columns=pairedColumns)
pairedCount = 0

#Puts stock into dataframes based on their company and BUY/SELL status
for i in range(len(symbols)):
    #Dataframe with Buy and Sell order
    d['t' + str(i)] = df[(df['Symbol'] == symbols[i])]
    d['t' + str(i)].reset_index(inplace=True, drop=True)
    d['t' + str(i)].reset_index(inplace=True)
    #Buy dataframe is sorted by price with lowest price in first position
    d['b' + str(i)] = d['t' + str(i)][d['t' + str(i)]['Buy/Sell'] == 'BUY']
    #Sell dataframe is sorted by date with oldest date in first position
    d['s' + str(i)] = d['t' + str(i)][d['t' + str(i)]['Buy/Sell'] == 'SELL']
    
    #Removes SELLs
    d['t' + str(i)] = d['t' + str(i)].drop(d['t' + str(i)][d['t' + str(i)]['Buy/Sell'] == 'SELL'].index)
    
    #Give dataframes their own index
    d['s' + str(i)].reset_index(inplace=True)
    d['t' + str(i)].reset_index(inplace=True, drop=True)
    
    #Sorts by price
    d['b' + str(i)] = d['b' + str(i)].sort_values(by="Price")
    d['b' + str(i)].reset_index(inplace=True, drop=True)
    
    #Adds column to track lost profits
    d['b' + str(i)]['ProfitsLost'] = 0
    
    #Loops through each SELL stock to find its corresponding buy stock
    for sell_count in range(d['s' + str(i)].shape[0]):
        
        #Stores date stock sold    
        sell_date = d['s' + str(i)].loc[sell_count, 'TradeDate']
        #Stores quantity of stock sold
        sell_quan = d['s' + str(i)].loc[sell_count, 'Quantity']
        #Stores index
        sell_index = d['s' + str(i)].loc[sell_count, 'index']
        
        #Manually set index for BUY dataframe
        buy_count = 0
        #Updates number of entries in BUY dataframe in case any have been removed
        buy_size = d['b' + str(i)].shape[0]

        if buy_size == 0:
            print('\nWARNING: The following stock has no corresponding BUY in report:\n', d['s'+str(i)].iloc[sell_count],'\n')
        
        #Loops through the BUY dataframe to match with SELL stock
        while buy_count < buy_size:
            #Stores date stock was bought
            buy_date = d['b' + str(i)]['TradeDate'][buy_count]
            #Stores quantity of stock bought
            buy_quan = d['b' + str(i)]['Quantity'][buy_count]
            #Stores index
            buy_index = d['b' + str(i)]['index'][buy_count]
            
            #Checks if stock was sold after stock was bought and if quantity is the same
            if ((buy_date <= sell_date) & (buy_quan == abs(sell_quan)) & (sell_index > buy_index)):
                #Sets up row values
                assetClass = d['b' + str(i)].loc[buy_count, 'AssetClass']
                symbol = d['b' + str(i)].loc[buy_count, 'Symbol']
                buyPrice = d['b' + str(i)].loc[buy_count, 'Price']
                sellPrice = d['s' + str(i)].loc[sell_count, 'Price']
                buyCommission = d['b' + str(i)].loc[buy_count, 'Commission']
                sellCommission = d['s' + str(i)].loc[sell_count, 'Commission']
                daysHeld = (sell_date - buy_date).days
                profitLost = d['b' + str(i)].loc[buy_count, 'ProfitsLost']
                
                #Calculates profit with ProfitLost
                if (d['b' + str(i)].loc[buy_count, 'ProfitsLost'] != 0):
                    profit = round((d['b' + str(i)].loc[buy_count, 'NetCash'] + d['s' + str(i)].loc[sell_count, 'NetCash']), 2)
                    profit = profit + d['b' + str(i)].loc[buy_count, 'ProfitsLost']
                else:
                    profit = round((d['b' + str(i)].loc[buy_count, 'NetCash'] + d['s' + str(i)].loc[sell_count, 'NetCash']), 2)
                
                #Adds paired data to dataframe
                pairedResults.loc[pairedCount] = [assetClass, symbol, buy_date, sell_date, buy_quan, buyPrice, sellPrice,
                                                 buyCommission, sellCommission, profit, daysHeld, profitLost]
                pairedCount = pairedCount + 1
                
                #Checks if profit is negative
                if (profit < 0):
                    #Finds the next BUY after current SELL
                    for total_index in range(d['t' + str(i)].shape[0]):
                        dt_index = d['t' + str(i)].loc[total_index, 'index']
                        if (dt_index > sell_index):
                            #Adds negative profit to ProfitsLost of next BUY
                            d['b' + str(i)].loc[d['b' + str(i)]['index'] == dt_index, 'ProfitsLost'] += profit
                            break                           
                
                #Drops BUY stock from dataframe to be printed
                d['t' + str(i)].drop(d['t' + str(i)][d['t' + str(i)]['index'] == buy_index].index)
                d['b' + str(i)].drop([buy_count], inplace = True)
                #Updates index
                d['b' + str(i)].reset_index(drop=True,inplace=True)
                #Update count to exit while loop to move to next SELL stock
                buy_count = buy_size + 1

            #Checks if stock was sold after stock was bought and if BUY quantity is larger than SELL quantity
            elif ((buy_date <= sell_date) & (buy_quan > abs(sell_quan)) & (sell_index > buy_index)):
                #Sets up row values
                assetClass = d['b' + str(i)].loc[buy_count, 'AssetClass']
                symbol = d['b' + str(i)].loc[buy_count, 'Symbol']
                buyPrice = d['b' + str(i)].loc[buy_count, 'Price']
                sellPrice = d['s' + str(i)].loc[sell_count, 'Price']
                buyCommission = d['b' + str(i)].loc[buy_count, 'Commission']
                sellCommission = d['s' + str(i)].loc[sell_count, 'Commission']
                daysHeld = (sell_date - buy_date).days
                profitLost = d['b' + str(i)].loc[buy_count, 'ProfitsLost']
                
                #Calculates profit with ProfitLost
                if (d['b' + str(i)].loc[buy_count, 'ProfitsLost'] != 0):
                    profit = round((((sell_quan * buyPrice) + buyCommission) + d['s' + str(i)].loc[sell_count, 'NetCash']), 2)
                    profit = profit + d['b' + str(i)].loc[buy_count, 'ProfitsLost']
                    d['b' + str(i)].loc[buy_count, 'ProfitsLost'] = 0
                else:
                    profit = round((((sell_quan * buyPrice) + buyCommission) + d['s' + str(i)].loc[sell_count, 'NetCash']), 2)
                    
                #Adds paired data to dataframe    
                pairedResults.loc[pairedCount] = [assetClass, symbol, buy_date, sell_date, abs(sell_quan), buyPrice, sellPrice,
                                                 buyCommission, sellCommission, profit, daysHeld, profitLost]
                pairedCount = pairedCount + 1
                
                #Checks if profit is negative
                if (profit < 0):
                    #Finds the next BUY after current SELL
                    for total_index in range(d['t' + str(i)].shape[0]):
                        dt_index = d['t' + str(i)].loc[total_index, 'index']
                        if (dt_index > sell_index):
                            #Adds negative profit to ProfitsLost of next BUY
                            d['b' + str(i)].loc[d['b' + str(i)]['index'] == dt_index, 'ProfitsLost'] += profit
                            break 
                
                
                #Updates BUY quantity and netcash based on number of SELL
                d['b' + str(i)].loc[buy_count, 'Quantity'] = buy_quan + sell_quan
                d['b' + str(i)].loc[buy_count, 'NetCash'] = round(((-(buy_quan + sell_quan) * buyPrice) + buyCommission), 3)
                #Update count to exit while loop to move to next SELL stock
                buy_count = buy_size + 1
            
            #Checks if stock was sold after stock was bought and if BUY quantity is less than SELL quantity
            #Will continue to loop in BUY dataframe to match rest of SELL
            elif ((buy_date <= sell_date) & (buy_quan < abs(sell_quan)) & (sell_index > buy_index)):
                #Sets up row values
                assetClass = d['b' + str(i)].loc[buy_count, 'AssetClass']
                symbol = d['b' + str(i)].loc[buy_count, 'Symbol']
                buyPrice = d['b' + str(i)].loc[buy_count, 'Price']
                sellPrice = d['s' + str(i)].loc[sell_count, 'Price']
                buyCommission = d['b' + str(i)].loc[buy_count, 'Commission']
                sellCommission = d['s' + str(i)].loc[sell_count, 'Commission']
                profit = round((d['b' + str(i)].loc[buy_count, 'NetCash'] + ((buy_quan * sellPrice) + sellCommission)), 2)
                daysHeld = (sell_date - buy_date).days
                profitLost = d['b' + str(i)].loc[buy_count, 'ProfitsLost']
                
                #Calculates profit with ProfitsLost
                if (d['b' + str(i)].loc[buy_count, 'ProfitsLost'] != 0):
                    profit = round((d['b' + str(i)].loc[buy_count, 'NetCash'] + ((buy_quan * sellPrice) + sellCommission)), 2)
                    profit = profit + d['b' + str(i)].loc[buy_count, 'ProfitsLost']
                else:
                    profit = round((d['b' + str(i)].loc[buy_count, 'NetCash'] + ((buy_quan * sellPrice) + sellCommission)), 2)
                
                #Adds paired data to dataframe
                pairedResults.loc[pairedCount] = [assetClass, symbol, buy_date, sell_date, buy_quan, buyPrice, sellPrice,
                                                 buyCommission, sellCommission, profit, daysHeld, profitLost]
                pairedCount = pairedCount + 1
                
                #Checks if profit is negative
                if (profit < 0):
                    #Finds the next BUY after current SELL
                    for total_index in range(d['t' + str(i)].shape[0]):
                        dt_index = d['t' + str(i)].loc[total_index, 'index']
                        if (dt_index > sell_index):
                            #Adds negative profit to ProfitsLost of next BUY
                            d['b' + str(i)].loc[d['b' + str(i)]['index'] == dt_index, 'ProfitsLost'] += profit
                            break 
                
                #Updates the SELL quantity based on number of BUY
                d['s' + str(i)].loc[sell_count, 'Quantity'] = buy_quan + sell_quan
                sell_quan = buy_quan + sell_quan
                d['s' + str(i)].loc[sell_count, 'NetCash'] = round(((-sell_quan * sellPrice) + sellCommission), 3)
                #Drops BUY stock from dataframe to be printed
                d['b' + str(i)].drop([buy_count], inplace = True)
                d['t' + str(i)].drop(d['t' + str(i)][d['t' + str(i)]['index'] == buy_index].index)
                #Update index
                d['b' + str(i)].reset_index(drop=True,inplace=True)
                
            #Will print statement if SELL goes through BUY dataframe without finding a match
            elif ((buy_count == (buy_size-1))):
                print('Error: No BUY stock available for SELL stock with given parameters')
                print('Stock info:', d['s' + str(i)].iloc[sell_count])
                buy_count = buy_size + 1

            #Checks next BUY if there is still SELL remaining
            else:
                buy_count = buy_count + 1

            #Manually update size of BUY dataframe and quantity to SELL before looping    
            buy_size = d['b' + str(i)].shape[0]
            

    #Adds the remaining BUY stocks in to one dataframe
    results = pd.concat([results, d['b' + str(i)]], ignore_index = True)

#Format columns for print
results['TradeDate'] = results['TradeDate'].astype(str)
pairedResults['BuyDate'] = pairedResults['BuyDate'].astype(str)
pairedResults['SellDate'] = pairedResults['SellDate'].astype(str)
results.drop(columns='index', inplace=True)
results['ProfitsLost'].replace(0, np.nan, inplace=True)
pairedResults['ProfitsLost'].replace(0, np.nan, inplace=True)
    
###REQUIRED: Change to your desired FilePath & FileName
FilePathOut = 'Downloads'
FileNameOut = 'StockReport.xlsx'

with pd.ExcelWriter(FilePathOut+'/'+FileNameOut) as writer:  
    pairedResults.to_excel(writer, sheet_name='Paired Results', index=False)
    results.to_excel(writer, sheet_name='Remaining Stocks', index=False)
    
print(FileNameOut, "generated into", FilePathOut)

Hold for Request.
First report obtained successfully
Hold for Request.
Second report obtained successfully
StockReport.xlsx generated into Downloads
